# LeNet Lab
![LeNet Architecture](lenet.png)
Source: Yan LeCun

## Load Data

Load the MNIST data, which comes pre-loaded with TensorFlow.

You do not need to modify this section.

In [1]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = "train.p"
validation_file= "valid.p"
testing_file = "test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

print()
print("Image Shape: {}".format(X_train[0].shape))
print()
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_valid)))
print("Test Set:       {} samples".format(len(X_test)))


Image Shape: (32, 32, 3)

Training Set:   34799 samples
Validation Set: 4410 samples
Test Set:       12630 samples


The MNIST data that TensorFlow pre-loads comes as 28x28x1 images.

However, the LeNet architecture only accepts 32x32xC images, where C is the number of color channels.

In order to reformat the MNIST data into a shape that LeNet will accept, we pad the data with two rows of zeros on the top and bottom, and two columns of zeros on the left and right (28+2+2 = 32).

You do not need to modify this section.

In [2]:
import numpy as np


In [3]:
#print(X_train[0], type(X_train[0]) )

def vecI(vec):
    return (vec-128.)/128



X_train = [vecI(i) for i in X_train ]
X_valid = [vecI(i) for i in X_valid ]
X_test = [vecI(i) for i in X_test ]
#print(X_train[0])


## Visualize Data

View a sample from the dataset.

You do not need to modify this section.

In [4]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

index = random.randint(0, len(X_train))
image = X_train[index].squeeze()

plt.figure(figsize=(1,1))
plt.imshow(image)
print(y_train[index])

22
Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x0000009CDE768EA0> (for post_execute):


ValueError: Floating point image RGB values must be in the 0..1 range.

ValueError: Floating point image RGB values must be in the 0..1 range.

## Preprocess Data

Shuffle the training data.

You do not need to modify this section.

In [5]:
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)

## Setup TensorFlow
The `EPOCH` and `BATCH_SIZE` values affect the training speed and model accuracy.

You do not need to modify this section.

In [6]:
import tensorflow as tf

EPOCHS = 10
BATCH_SIZE = 512

## TODO: Implement LeNet-5
Implement the [LeNet-5](http://yann.lecun.com/exdb/lenet/) neural network architecture.

This is the only cell you need to edit.
### Input
The LeNet architecture accepts a 32x32xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

### Architecture
**Layer 1: Convolutional.** The output shape should be 28x28x6.

**Activation.** Your choice of activation function.

**Pooling.** The output shape should be 14x14x6.

**Layer 2: Convolutional.** The output shape should be 10x10x16.

**Activation.** Your choice of activation function.

**Pooling.** The output shape should be 5x5x16.

**Flatten.** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D. The easiest way to do is by using `tf.contrib.layers.flatten`, which is already imported for you.

**Layer 3: Fully Connected.** This should have 120 outputs.

**Activation.** Your choice of activation function.

**Layer 4: Fully Connected.** This should have 84 outputs.

**Activation.** Your choice of activation function.

**Layer 5: Fully Connected (Logits).** This should have 10 outputs.

### Output
Return the result of the 2nd fully connected layer.

In [7]:
from tensorflow.contrib.layers import flatten

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # TODO: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    weight_layer_1 = tf.Variable(tf.truncated_normal((5,5, 3, 6),mu,sigma))
    bias_1 = tf.Variable(tf.zeros(6))
    # TODO: Activation.
    layer_1 = tf.nn.conv2d(x, weight_layer_1, [1,1,1,1], padding='VALID') + bias_1
    layer_1 = tf.nn.relu(layer_1)
    
    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.   
    # can also do stride = 2 with padding SAME 
    
    layer_1 = tf.nn.max_pool(layer_1,
                             ksize=[1, 2,2, 1],
                             strides=[1, 2,2, 1],
                           padding='VALID')
    
    
    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    
    weight_layer_2 = tf.Variable(tf.truncated_normal((5,5, 6, 16),mu,sigma))
    bias_2 = tf.Variable(tf.zeros(16))
    layer_2 = tf.nn.conv2d(layer_1, weight_layer_2, 
                           strides=[1, 1, 1, 1], padding='VALID') + bias_2
    layer_2 = tf.nn.relu(layer_2)
    
    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    layer_2 = tf.nn.max_pool(layer_2,
                             ksize=[1, 2,2, 1],
                             strides=[1,2,2, 1],
                            padding='VALID')
    
    # TODO: Flatten. Input = 5x5x16. Output = 400.  
    
    layer_2 = flatten(layer_2)    
    
    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    #print(layer_2.shape)
    weight_layer_3 = tf.Variable(tf.truncated_normal((400,120),mu,sigma))
    bias_3 = tf.Variable(tf.zeros(120) )
    l3_fc1 = tf.matmul(layer_2, weight_layer_3) + bias_3
    
    # TODO: Activation.
    l3_fc1 = tf.nn.relu(l3_fc1)
    
    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    weight_layer_4 = tf.Variable(tf.truncated_normal((120,84),mu,sigma))
    bias_4 = tf.Variable(tf.zeros(84))
    l4_fc2 = tf.matmul(l3_fc1, weight_layer_4) + bias_4
    
    # TODO: Activation.
    l4_fc2 = tf.nn.relu(l4_fc2)
    #l4_fc2 = tf.nn.dropout(l4_fc2, dropout)
    #print(l4_fc2.shape)

    # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
    weight_layer_5 = tf.Variable(tf.truncated_normal((84,43),mu,sigma))
    bias_5 = tf.Variable(tf.zeros(43))    
    logits = tf.matmul(l4_fc2, weight_layer_5) +  bias_5
    # TODO: Activation.
    
    return logits

x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)

rate = 0.001

logits = LeNet(x)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

## Features and Labels
Train LeNet to classify [MNIST](http://yann.lecun.com/exdb/mnist/) data.

`x` is a placeholder for a batch of input images.
`y` is a placeholder for a batch of output labels.

You do not need to modify this section.

## Training Pipeline
Create a training pipeline that uses the model to classify MNIST data.

You do not need to modify this section.

## Model Evaluation
Evaluate how well the loss and accuracy of the model for a given dataset.

You do not need to modify this section.

In [8]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

## Train the Model
Run the training data through the training pipeline to train the model.

Before each epoch, shuffle the training set.

After each epoch, measure the loss and accuracy of the validation set.

Save the model after training.

You do not need to modify this section.

In [9]:
import time

start = time.time()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")
fina = time.time()
print("it took", (fina-start)," to train")

Training...

EPOCH 1 ...
Validation Accuracy = 0.431

EPOCH 2 ...
Validation Accuracy = 0.689

EPOCH 3 ...
Validation Accuracy = 0.761

EPOCH 4 ...
Validation Accuracy = 0.804

EPOCH 5 ...
Validation Accuracy = 0.831

EPOCH 6 ...
Validation Accuracy = 0.838

EPOCH 7 ...
Validation Accuracy = 0.859

EPOCH 8 ...
Validation Accuracy = 0.857

EPOCH 9 ...
Validation Accuracy = 0.862

EPOCH 10 ...
Validation Accuracy = 0.864

Model saved
it took 289.83835220336914  to train


## Evaluate the Model
Once you are completely satisfied with your model, evaluate the performance of the model on the test set.

Be sure to only do this once!

If you were to measure the performance of your trained model on the test set, then improve your model, and then measure the performance of your model on the test set again, that would invalidate your test results. You wouldn't get a true measure of how well your model would perform against real data.

You do not need to modify this section.

In [10]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

INFO:tensorflow:Restoring parameters from .\lenet
Test Accuracy = 0.874
